In [106]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sort import Sort

In [107]:
def detect_red_hough(board):
    img_hsv=cv2.cvtColor(board, cv2.COLOR_RGB2HSV)

    # lower mask (0-10)
    lb = 60 #120
    lower_red = np.array([0,lb,lb])
    upper_red = np.array([9,255,255])
    mask0 = cv2.inRange(img_hsv, lower_red, upper_red)

    # upper mask (170-180)
    lower_red = np.array([160,lb,lb])
    upper_red = np.array([180,255,255])
    mask1 = cv2.inRange(img_hsv, lower_red, upper_red)

    k1 = np.ones((5,5))

    # join my masks
    redmask = mask0+mask1
    redmask = cv2.erode(redmask, k1)
    #redmask = cv2.erode(redmask, k1)
    #redmask = cv2.erode(redmask, k1)
    redmask = cv2.dilate(redmask, k1)
    redmask = cv2.dilate(redmask, k1)
    redmask = cv2.dilate(redmask, k1)
    redmask = cv2.dilate(redmask, k1)
    #redmask = cv2.dilate(redmask, k1)



    redboard = np.array(board)
    redboard[redmask==0] = 0
    redboard = cv2.cvtColor(redboard, cv2.COLOR_RGB2GRAY)
    redboard = cv2.equalizeHist(redboard)

    # redboard = cv2.cvtColor(redboard, cv2.COLOR_RGB2GRAY)

    minDist = 30
    param1 = 80 #canie threshold
    param2 = 28 #200 #smaller value-> more false circles
    minRadius = 30
    maxRadius = 40 #10

    circles = cv2.HoughCircles(redboard,cv2.HOUGH_GRADIENT, 1.2, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)
    return circles,redboard

# def detect_white_hough(board):
#     epsilon = 90 #190
#     white_upper_range = np.array([255,255,255])
#     white_lower_range = white_upper_range - epsilon

#     k1 = np.ones((15,15))
#     k2 = np.ones((7,7))
    

#     whitemask = cv2.inRange(board, white_lower_range, white_upper_range)
#     whitemask = cv2.erode(whitemask, k1)
#     whitemask = cv2.erode(whitemask, k1)


#     #whitemask = cv2.dilate(whitemask ,k)
#     #whitemask = cv2.dilate(whitemask ,k)
#     whitemask = cv2.dilate(whitemask, k2)
#     whitemask = cv2.dilate(whitemask, k2)

#     whiteboard = np.array(board)
#     whiteboard[whitemask==0] = 0
#     whiteboard = cv2.cvtColor(whiteboard, cv2.COLOR_RGB2GRAY)

#     minDist = 30
#     param1 = 30 #500
#     param2 = 11 #30 #smaller value-> more false circles
#     minRadius = 30
#     maxRadius = 40 #10

#     #circles = cv2.HoughCircles(whiteboard,cv2.HOUGH_GRADIENT, 1.2, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)
#     circles = cv2.HoughCircles(whitemask,cv2.HOUGH_GRADIENT, 1.2, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)
#     return circles,whitemask


def detect_white_hough(board):
    epsilon = 36 #34 with equalization
    white_upper_range = np.array([255])
    white_lower_range = white_upper_range - epsilon

    k1 = np.ones((11,11))
    k2 = np.ones((7,7))
    
    board_ = cv2.cvtColor(board, cv2.COLOR_RGB2GRAY)
    board_ = cv2.equalizeHist(board_)

    whitemask = cv2.inRange(board_, white_lower_range, white_upper_range)

    whitemask = cv2.erode(whitemask, k1)
    #whitemask = cv2.erode(whitemask, k1)
    #whitemask = cv2.erode(whitemask, k2)

    whitemask = cv2.dilate(whitemask,k2)
    whitemask = cv2.dilate(whitemask,k2)
    whitemask = cv2.dilate(whitemask, k2)
    whitemask = cv2.dilate(whitemask, k2)

    whiteboard = np.array(board)
    whiteboard[whitemask==0] = 0
    whiteboard = cv2.cvtColor(whiteboard, cv2.COLOR_RGB2GRAY)

    minDist = 30
    param1 = 60 #60
    param2 = 25 #25 #smaller value-> more false circles
    minRadius = 30
    maxRadius = 40 #10

    circles = cv2.HoughCircles(whiteboard,cv2.HOUGH_GRADIENT, 1, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)
    #circles = cv2.HoughCircles(whitemask,cv2.HOUGH_GRADIENT, 1.2, minDist, param1=param1, param2=param2, minRadius=minRadius, maxRadius=maxRadius)
    return circles,whitemask

def detect_dice(board):
    img_hsv=cv2.cvtColor(board, cv2.COLOR_RGB2HSV)
    params = cv2.SimpleBlobDetector_Params()

    # Change thresholds
    params.minThreshold = 40
    params.maxThreshold = 200

    # Filter by Area.
    params.filterByArea = True
    params.minArea = 10

    # Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0.5

    # Filter by Convexity
    params.filterByConvexity = False
    params.minConvexity = 0.87

    # Filter by Inertia
    params.filterByInertia = False
    params.minInertiaRatio = 0.01

    detector = cv2.SimpleBlobDetector_create(params)

    k = np.ones((3,3))
    lb = 60
    lower_red = np.array([90,lb,lb])
    upper_red = np.array([170,255,255])
    mask0 = cv2.inRange(img_hsv, lower_red, upper_red)
    #mask0 = cv2.dilate(mask0, k)

    cnts = cv2.findContours(mask0, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    dice = []
    min_area = 700
    max_area = 1400
    for c in cnts:
        area = cv2.contourArea(c)
        if area > min_area and area < max_area:
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)
            box = box.astype(np.int64)
            x,y,w,h = cv2.boundingRect(c)

            # Width and height of the rectangle
            width = int(rect[1][0])
            height = int(rect[1][1])

            # Define target square size (max of width and height)
            size = max(width, height)

            # Destination points for warping
            dst_pts = np.array([[0, 0], [size - 1, 0], [size - 1, size - 1], [0, size - 1]], dtype="float32")

            # Compute the perspective transform matrix and apply it
            M = cv2.getPerspectiveTransform(np.float32(box), dst_pts)
            warped = cv2.warpPerspective(mask0, M, (size, size))
            keypoints = detector.detect(warped)
            dice.append([len(keypoints), (x,y,w,h)])

            # im_with_keypoints = cv2.drawKeypoints(warped, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

            # plt.imshow(im_with_keypoints)
            # plt.show()

            # plt.imshow(board[y:y+h, x:x+w])
            # plt.show()

            # x,y,w,h = cv2.boundingRect(c)
            # dice = mask0[y:y+h, x:x+w]

            # plt.imshow(dice)
            # plt.show()

            # dice_cnts = cv2.findContours(dice, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            # dice_cnts = dice_cnts[0] if len(dice_cnts) == 2 else dice_cnts[1]

            # print(len(dice_cnts))
    return dice


In [108]:
%matplotlib inline
cap = cv2.VideoCapture('videos/easy1.mp4')

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
size = 500
out1 = cv2.VideoWriter('output_with_dice.mp4', fourcc, 30.0, (size,  size))
# out2 = cv2.VideoWriter('output_redmask.mp4', fourcc, 30.0, (size,  size))
# out3 = cv2.VideoWriter('output_whitemask.mp4', fourcc, 30.0, (size,  size))


red_tracker = Sort(max_age=620, iou_threshold=0.00001, min_hits=3)

white_tracker = Sort(max_age=620, iou_threshold=0.00001, min_hits=3)
j = 0
 
while cap.isOpened():
    j+=1
    ret, frame = cap.read()

    # if j < 500:
    #     continue
    # if j > 500:
    #     break
 
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    frame = cv2.resize(frame, (1000,1000))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    dices = detect_dice(frame)
    for dice in dices:
        score, (x,y,w,h) = dice
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        cv2.putText(frame, f'{score}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    red_pawns, redboard = detect_red_hough(frame)
    red_detections = [[x-35, y-35, x + 35, y + 35, 1] for x, y, r in red_pawns[0, :]]

    red_tracked = red_tracker.update(np.array(red_detections))

    for obj in red_tracked:
        obj = np.uint16(np.around(obj))
        x_min, y_min, x_max, y_max, obj_id = obj

        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
        cv2.putText(frame, f'ID: {int(obj_id)}', (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    white_pawns, whitemask = detect_white_hough(frame)
    white_detections = [[x-35, y-35, x + 35, y + 35, 1] for x, y, r in white_pawns[0, :]]

    white_tracked = white_tracker.update(np.array(white_detections))

    for obj in white_tracked:
        obj = np.uint16(np.around(obj))
        x_min, y_min, x_max, y_max, obj_id = obj

        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 255, 255), 2)
        cv2.putText(frame, f'ID: {int(obj_id)}', (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    frame = cv2.resize(frame, (size,size))
    out1.write(frame)

    # redboard = cv2.resize(redboard, (size,size))
    # redboard = cv2.cvtColor(redboard, cv2.COLOR_GRAY2BGR)
    # out2.write(redboard)

    # whitemask = cv2.resize(whitemask, (size,size))S
    # whitemask = cv2.cvtColor(whitemask, cv2.COLOR_G RAY2BGR)
    # out3.write(whitemask)
 
print('done')
cap.release()
out1.release()
# out2.release()
# out3.release()

Can't receive frame (stream end?). Exiting ...
done
